In [59]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import time

# Load your pre-trained classifier model (adjust the path to your model)
model = load_model('asl_resnet.h5',compile = False)

# Class labels for ASL (A-Z, space, delete, etc.)

# Set up camera
cap = cv2.VideoCapture(0)
class_labels = {i: chr(65 + i) for i in range(26)}
class_labels[26] = 'Delete'
class_labels[27] = 'Space'
class_labels[28] = 'Nothing'


last_prediction_time = time.time()
# Function to preprocess the image before sending it to the classifier
def preprocess_image(image, size=(64, 64)):  # Adjust size as per your model's input
    image = cv2.resize(image, size)
    image = np.array(image, dtype='float32')
    image = image / 255.0  # Normalize if your model requires it
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Function to draw a rectangle (demarcated area)
def draw_demarcation(frame):
    height, width, _ = frame.shape
    x_start, y_start, x_end, y_end = int(width * 0.3), int(height * 0.2), int(width * 0.7), int(height * 0.8)
    cv2.rectangle(frame, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)
    return frame, (x_start, y_start, x_end, y_end)
def check_condition(recognized_text, recognized_letter):
    if recognized_letter =="Space" :
        recognized_text.append(" ")
    elif recognized_letter == "Delete":
        recognized_text = recognized_text[:-7]

    return recognized_text
        
# Main loop
recognized_text = ''
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Flip the frame horizontally for a mirror effect
    frame = cv2.flip(frame, 1)

    # Draw the demarcated area
    frame, (x_start, y_start, x_end, y_end) = draw_demarcation(frame)

    # Extract the region of interest (ROI) for classification
    roi = frame[y_start:y_end, x_start:x_end]
    # Preprocess the ROI
    
    if time.time() - last_prediction_time >= 2:
        processed_roi = preprocess_image(roi)
        # Use the classifier to predict the gesture
        prediction = model.predict(processed_roi)
        predicted_class = np.argmax(prediction)
        recognized_letter = class_labels[predicted_class]
    
        # Add recognized letter to the text
        recognized_text += recognized_letter
        recognized_text = check_condition(recognized_text,recognized_letter)
        last_prediction_time = time.time()

    # Display the actual camera feed with the demarcated area
    cv2.imshow('Camera Feed', frame)
    
    # Display the recognized text in a separate window
    text_frame = np.zeros((300, 600, 3), dtype='uint8')  # Black background
    cv2.putText(text_frame, f'Recognized Text: {recognized_text}', (30, 150),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow('Recognized Text', text_frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()
